# Vertical mixing in numerical models

## As a mixing time scale

### Mathematical background

Imagine a two box system with unequal concentrations. Call them $c_1$ and $c_2$. To build a conceptual model, let's imagine that the gradient between the two boxes decays according to first order kinetics, that is:

$$ \frac{\partial}{\partial t}(c_1 - c_2) = -k(c_1 - c_2) $$

This assumes that the gradient follows an exponential decay in time. This makes a certain amount of intuitive sense, as we'd expect larger gradients to lead to faster changes in concentration to equalize than would smaller gradients.

To calculate the change in $c_1$ and $c_2$, we distribute the derivative:

$$ \frac{\partial c_1}{\partial t} - \frac{\partial c_2}{\partial t} = -k(c_1 - c_2) $$

Then we must recognize that in order to satisfy mass conservation, $\partial c_1/\partial t = -\partial c_2/\partial t$, which gives us:

$$
2 \frac{\partial c_1}{\partial t} = -k(c_1 - c_2) \\ 
\frac{\partial c_1}{\partial t} = -\frac{\partial c_2}{\partial t} = -\frac{k}{2}(c_1 - c_2) 
$$

Check that the signs agree with our intuition: if $c_1 > c_2$, $\partial c_1/\partial c_2$ should be $< 0$, which is the case here.

Thus, in order to calcuate the values of $c_1$ and $c_2$ at any time $t$, we need to solve these differential equations. Unlike a normal first order decay, these don't have nice simple solutions because it is the difference, not the individual concentrations, that matter. We do know how the difference between the boxes should vary though - that _does_ follow a simple exponential decay.

Let $c_1 - c_2 = \Delta c$. Then (switching to regular derivatives instead of partial because now we only have two variables):
$$
\frac{d\Delta c}{dt} = -k\Delta c \\
\Rightarrow \Delta c(t) = \Delta c(t_0) e^{-kt}
$$


### Programmatic implementation

The analytical solution for this two-box problem is easy to solve, but hard to generalize to arbitrarily many boxes. That's why numerical methods are so valuable; they are much easier to expand to an arbitrary number of boxes. Generally, numerical methods work by evaluating the derivatives at the current time, then using that derivative value to project how each quantity will change over some fixed time step. The time step must be small enough that the derivative does not change much over it's duration.

A simple implementation of the above equations (in Julia, which is kind of similar to Matlab) looks like:

In [79]:
using Plots;

dt = 0.1;  # time step = 0.1 second
run_time = 50;  # how long the model should simulate in seconds
nsteps = convert(Int64, run_time/dt);  # how many steps the model needs to take. 
                                       # Julia is a bit picky about types compared to Matlab, so convert from float 
                                       # to integer (or the indexing in the for loop fails)
c = zeros(nsteps+1, 2);  # set up an array to hold the concentration in both boxes over the modeled time
c[1,1] = 10.0;  # initial condition: box 1 has a concentration of 10.0 (arbitrary units) and box 2 has nothing
k = 0.1;  # mixing rate constant in s^{-1}
for n=1:nsteps
    dc_dt = -k*(c[n,1] - c[n,2]);  # calculate the change in concentration at timestep i
    c[n+1, 1] = c[n, 1] + dc_dt * dt;  # calculate the box 1 concentration at the next timestep, 
                                       # assuming dc_dt is constant over the timestep
    c[n+1, 2] = c[n, 2] - dc_dt * dt;  # calculate the box 2 concentration at the next timestep, 
                                       # remembering dc_2/dt = - dc_1/dt
end

time_vector = (0.0:(nsteps+1))*dt;
plot(time_vector, c[:,1], label="Box 1", xlabel="Time (s)", ylabel="Concentration (arbitrary units)")
plot!(time_vector, c[:,2], label="Box 2")

As we'd expect, the concentrations in the two boxes converge. How does this compare to the analytical solution? Well, we only have an analytical solution for the difference between the boxes, so that's what we'll compare:

In [80]:
delta_c = c[:,1] - c[:,2]
analytical_soln = delta_c[1] * e.^(-k * time_vector)
plot(time_vector, delta_c, label="Numerical solution", xlabel="Time (s)", ylabel="Delta C")
plot!(time_vector, analytical_soln, label="Analytical solution")

The numerical solution converges faster than the analytical one. This makes a certain sense - because the numerical solution assumes that the $d\Delta c/dt$ stays constant over each time step, and each $d\Delta c/dt$ is computed at the beginning of it's timestep, the change is overestimated.

### More boxes and the diffusion equation

Having only two boxes is boring - what about adding more? For now, we'll stick to a 1D model, meaning that each box has two neighbors - top and bottom, except the first and last boxes obviously. For now, let's focus on the boxes that do have two neighbors. Amending the solution to allow for multiple boxes is easy, we just need to sum the contributions from both neighbors. We'll use a more general index notation now, using a subscript $i$ to indicate the $i$-th box:

$$ \frac{\partial c_i}{\partial t} = -k(c_i - c_{i-1}) - k(c_i - c_{i+1}) $$

In [91]:
using Plots;

dt = 0.1;  # time step = 0.1 second
run_time = 500;  # how long the model should simulate in seconds
nsteps = convert(Int64, run_time/dt);  # how many steps the model needs to take. 
                                       # Julia is a bit picky about types compared to Matlab, so convert from float 
                                       # to integer (or the indexing in the for loop fails)
nboxes = 10;  # how many boxes to include in the model
c = zeros(nsteps+1, nboxes);  # set up an array to hold the concentration in both boxes over the modeled time
c[1,1] = 10.0;  # initial condition: box 1 has a concentration of 10.0 (arbitrary units) and box 2 has nothing
k = 0.1;  # mixing rate constant in s^{-1}
for n=1:nsteps
    for i=1:nboxes
        dc_dt = 0.0;
        if i > 1
            dc_dt += -k*(c[n,i] - c[n,i-1]); # calculate the change in concentration at timestep n due to the box 
                                             # below as long as we're not in the bottom box
        end
        if i < nboxes
            dc_dt += -k*(c[n,i] - c[n,i+1]); # calculate the change in concentration at timestep n due to the box 
                                             # above as long as we're not in the top box
        end
        c[n+1, i] = c[n, i] + dc_dt * dt;  # calculate the box i concentration at the next timestep, 
                                           # assuming dc_dt is constant over the timestep
    end
end

time_vector = (0.0:(nsteps+1))*dt;
plot(time_vector, c, xlabel="Time (s)", ylabel="Concentration (arbitrary units)")

As you can see, now we get more interesting behavior! The second box is particularly cool, as initially the gradient vs. the first box is large enough that it's gaining concentration faster than it's losing it. As time goes on though, all the boxes equilibrate around 1/10th of the original concentration, as you'd expect.

Now, let's take another look at the equation:

$$ \frac{\partial c_i}{\partial t} = -k(c_i - c_{i-1}) - k(c_i - c_{i+1}) $$

This is a difference of differences, which suggests that perhaps this mixing is represented by a second derivative? In fact it is, and the diffusion equation is:

$$ \frac{\partial c}{\partial t} = D \frac{\partial^2 c}{\partial x^2} $$

where $D$ is the diffusion constant. It has units of m$^2$ s$^{-1}$. (We've dropped the negative sign because the correct sign will come out of the second derivative.)